In [3]:
#wget ftp://ngs.sanger.ac.uk/production/ag1000g/phase1/AR3/variation/main/hdf5/ag1000g.phase1.ar3.pass.3L.h5


In [10]:
import h5py

import dask.array as da

In [17]:
h5_3L = h5py.File('ag1000g.phase1.ar3.pass.3L.h5')
samples = h5_3L['/3L/samples']
calldata_genotype = h5_3L['/3L/calldata/genotype']

In [13]:
calldata_genotype

<HDF5 dataset "genotype": shape (9643193, 765, 2), type "|i1">

In [18]:
samples

<HDF5 dataset "samples": shape (765,), type "|S8">